In [1]:
import sys; sys.path.insert(0,'..')
from lltk import *
# LLTK.cdb.clear()

In [10]:
def sync(self,progress=True,**kwargs):
    iterr = list(self.init_meta_csv())
    if progress: iterr = get_tqdm(iterr, desc=f'[{self.name}] Syncing texts to db')
    o=[]
    for i,(id,d) in enumerate(iterr):
        addr = self.get_addr(id)
        meta = safebool(just_meta_no_id(d))
        t=Text(addr,_cache=True,**meta)
        o.append(t)
    return o

BaseCorpus.sync=sync

def load(self):
    with self.cdb:
        tdoc = self.cdb.get(self.addr)
    if not tdoc:
        log.error('nothing found to load')
        return
    if tdoc.data:
        newmeta = deserialize_map(dict(tdoc.data))
        self._meta={**self._meta, **newmeta}
    if log: log('we all good')
    return

def cache(self,ometa={},force=False,**y):
    new = self.ensure_id({
        # **(old if old else {}),
        **(self.__meta if self.__meta else {}),
        **(self._meta if self._meta else {}),
        **(ometa if ometa else {})
    },allow_sep=False)
    o=self.cache_cdb(new)
    return o

def cache_cdb(self,ometa={},force=False,**y):
    dbd={}
    dbd['id']=self.addr
    dbd['corpus']=self.corpus.id
    dbd['author']=self.author
    dbd['title']=self.title
    dbd['year']=self.year
    dbd['data']=serialize_map(just_meta_no_id(ometa))
    with self.cdb: return self.doctype.create(**dbd)

BaseText.cache=cache
BaseText.cache_cdb=cache_cdb

In [11]:
C=Corpus('markmark')
with LLTK: C.sync()

[13:22:44] lltk.corpus.corpus.init_meta_csv( MarkMarkCorpus(markmark) )


[MarkMark] Syncing texts to db:   0%|          | 0/355 [00:00<?, ?it/s]

[13:22:44] lltk.corpus.corpus.text( <- id = Caldwell,_Erskine.Tobacco_Road, corpus = MarkMarkCorpus(markmark), kwargs = {'author': 'Caldwell, Erskine', 'corpus': 'MarkMark', 'dob': '1903.0', 'dod': '1987.0', 'gender': 'M) )
[13:22:44] __main__.cache( TextDoc <id=_markmark/Caldwell,_Erskine.Tobacco_Road> )


NameError: name 'stop' is not defined